# Cybersecurity Incident Prediction EDA for DSCI632 Project

David Blankenship: dwb65@drexel.edu

EDA Analysis Notebook for project. This was done in Google Colab If viewing in GitHub the .show() outputs will be off due to differences with how Google Colab and GitHub display it.

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

from pyspark.sql.functions import col, mean, min, max, count
import matplotlib.pyplot as plt

from pyspark.sql import SparkSession

spark = SparkSession.builder.appName('SparkByExamples.com').getOrCreate()

Mounted at /content/gdrive


In [ ]:
df = spark.read.csv('/content/gdrive/MyDrive/GUIDE_Train.csv', header=True, inferSchema = True)
df.show(5)

+-------------+-----+----------+-------+-------------------+----------+----------+-----------------+---------------+--------------+-------------+--------------+----------+------------+--------+------+---------+------+----------+----------+---------------+-----------+----------+----------------+--------------+-----------+-----------------+-----------------+-------------+---------------+------------------+------------+--------+----------+--------------+------------+-----+--------+---------+-----------------+--------------+-----------+-----------+-----+-----+
|           Id|OrgId|IncidentId|AlertId|          Timestamp|DetectorId|AlertTitle|         Category|MitreTechniques| IncidentGrade|ActionGrouped|ActionGranular|EntityType|EvidenceRole|DeviceId|Sha256|IpAddress|   Url|AccountSid|AccountUpn|AccountObjectId|AccountName|DeviceName|NetworkMessageId|EmailClusterId|RegistryKey|RegistryValueName|RegistryValueData|ApplicationId|ApplicationName|OAuthApplicationId|ThreatFamily|FileName|FolderPa

## Initial Inspection/Preprocessing

In [ ]:
print('Data Shape: ', (df.count(), len(df.columns)))

Data Shape:  (9516837, 45)


In [ ]:
df.printSchema()

root
 |-- Id: long (nullable = true)
 |-- OrgId: integer (nullable = true)
 |-- IncidentId: integer (nullable = true)
 |-- AlertId: integer (nullable = true)
 |-- Timestamp: timestamp (nullable = true)
 |-- DetectorId: integer (nullable = true)
 |-- AlertTitle: integer (nullable = true)
 |-- Category: string (nullable = true)
 |-- MitreTechniques: string (nullable = true)
 |-- IncidentGrade: string (nullable = true)
 |-- ActionGrouped: string (nullable = true)
 |-- ActionGranular: string (nullable = true)
 |-- EntityType: string (nullable = true)
 |-- EvidenceRole: string (nullable = true)
 |-- DeviceId: integer (nullable = true)
 |-- Sha256: integer (nullable = true)
 |-- IpAddress: integer (nullable = true)
 |-- Url: integer (nullable = true)
 |-- AccountSid: integer (nullable = true)
 |-- AccountUpn: integer (nullable = true)
 |-- AccountObjectId: integer (nullable = true)
 |-- AccountName: integer (nullable = true)
 |-- DeviceName: integer (nullable = true)
 |-- NetworkMessageId: i

In [ ]:
df.groupBy('IncidentGrade').count().show()

+--------------+-------+
| IncidentGrade|  count|
+--------------+-------+
|          NULL|  51340|
|BenignPositive|4110817|
| FalsePositive|2031967|
|  TruePositive|3322713|
+--------------+-------+



In [ ]:
51340/9516837 * 100

0.539464950382149

In [1]:
4110817/(4110817+2031967+3322713) * 100

43.42948922808807

In [2]:
2031967/(4110817+2031967+3322713) * 100

21.467092536186954

In [3]:
3322713/(4110817+2031967+3322713) * 100

35.10341823572497

We see that the nulls in the target is 0.5% of the overall dataset. As the dataset is already enormous I will probably just be dropping these. We also note that the targets are imbalanced necessitating an evaluation regime that accounts for this.

### Missing Data Inspection

In [ ]:
from pyspark.sql.functions import col, sum

null_counts = df.select([sum(col(c).isNull().cast("int")).alias(c) for c in df.columns])
null_counts.show()

+---+-----+----------+-------+---------+----------+----------+--------+---------------+-------------+-------------+--------------+----------+------------+--------+------+---------+---+----------+----------+---------------+-----------+----------+----------------+--------------+-----------+-----------------+-----------------+-------------+---------------+------------------+------------+--------+----------+--------------+------------+-------+--------+---------+-----------------+--------------+-----------+-----------+-----+----+
| Id|OrgId|IncidentId|AlertId|Timestamp|DetectorId|AlertTitle|Category|MitreTechniques|IncidentGrade|ActionGrouped|ActionGranular|EntityType|EvidenceRole|DeviceId|Sha256|IpAddress|Url|AccountSid|AccountUpn|AccountObjectId|AccountName|DeviceName|NetworkMessageId|EmailClusterId|RegistryKey|RegistryValueName|RegistryValueData|ApplicationId|ApplicationName|OAuthApplicationId|ThreatFamily|FileName|FolderPath|ResourceIdName|ResourceType|  Roles|OSFamily|OSVersion|Antispa

MitreTechniques, ActionGrouped, ActionGranular, EmailClusterId, ThreatFamily, ResourceType, Roles, AntispamDirection, SuspicionLevel, and LastVerdict lack enough data that I will be dropping these columns. I will look at MitreTechniques first.

Otherwise the dataset is mostly good aside from the target nulls we saw earlier.

In [ ]:
missing_cols = ['MitreTechniques', 'ActionGrouped', 'ActionGranular', 'EmailClusterId', 'ThreatFamily', 'ResourceType', 'Roles', 'AntispamDirection', 'SuspicionLevel', 'LastVerdict', 'IncidentGrade']

for c in missing_cols:
  print(c, ": ", df.filter(df[c].isNull()).count())

MitreTechniques :  5468386
ActionGrouped :  9460773
ActionGranular :  9460773
EmailClusterId :  9420025
ThreatFamily :  9441956
ResourceType :  9509762
Roles :  9298686
AntispamDirection :  9339535
SuspicionLevel :  8072708
LastVerdict :  7282572
IncidentGrade :  51340


In [ ]:
for c in missing_cols:
  print(c, " Missing Percent: ", (df.filter(df[c].isNull()).count()/df.count()) * 100)

MitreTechniques  Missing Percent:  57.4601204160584
ActionGrouped  Missing Percent:  99.41089670864385
ActionGranular  Missing Percent:  99.41089670864385
EmailClusterId  Missing Percent:  98.98272924081814
ThreatFamily  Missing Percent:  99.21317345248217
ResourceType  Missing Percent:  99.92565807316024
Roles  Missing Percent:  97.7077362993608
AntispamDirection  Missing Percent:  98.1369650441633
SuspicionLevel  Missing Percent:  84.8255360473233
LastVerdict  Missing Percent:  76.52302965785796
IncidentGrade  Missing Percent:  0.539464950382149


In [ ]:
df.groupBy('MitreTechniques').count().orderBy('count', ascending=False).show()

+--------------------+-------+
|     MitreTechniques|  count|
+--------------------+-------+
|                NULL|5468386|
|     T1078;T1078.004|1354904|
|           T1566.002| 814308|
|               T1566| 659591|
|               T1133| 145579|
|           T1566.001| 136892|
|T1110;T1110.003;T...|  88661|
|     T1087;T1087.002|  54564|
|               T1110|  41768|
|         T1078;T1098|  40342|
|T1559;T1106;T1059...|  37869|
|   T1046;T1071;T1210|  34229|
|         T1568;T1008|  33335|
|         T1110;T1078|  31570|
|         T1190;T1133|  31379|
|T1046;T1030;T1071...|  28769|
|               T1078|  27371|
|T1003;T1012;T1552...|  22170|
|               T1190|  20214|
|         T1071;T1048|  19526|
+--------------------+-------+
only showing top 20 rows



MITRE Techniques are a categorization of the specific methods involved in the cybersecurity incident.

https://attack.mitre.org/techniques/T1078/

The most common non-null value is a T1078 incident, which is an incident involving valid accounts. T1078.004 in particular is a cloud account which I find hilariously relevant to this class.

While valuable information the high volume of missing data likely means it has to be dropped. I am extremely torn on this though and suspect it would be best for future work. I will return to this and do some analysis on this column



In [ ]:
df.dtypes

[('Id', 'bigint'),
 ('OrgId', 'int'),
 ('IncidentId', 'int'),
 ('AlertId', 'int'),
 ('Timestamp', 'timestamp'),
 ('DetectorId', 'int'),
 ('AlertTitle', 'int'),
 ('Category', 'string'),
 ('MitreTechniques', 'string'),
 ('IncidentGrade', 'string'),
 ('ActionGrouped', 'string'),
 ('ActionGranular', 'string'),
 ('EntityType', 'string'),
 ('EvidenceRole', 'string'),
 ('DeviceId', 'int'),
 ('Sha256', 'int'),
 ('IpAddress', 'int'),
 ('Url', 'int'),
 ('AccountSid', 'int'),
 ('AccountUpn', 'int'),
 ('AccountObjectId', 'int'),
 ('AccountName', 'int'),
 ('DeviceName', 'int'),
 ('NetworkMessageId', 'int'),
 ('EmailClusterId', 'double'),
 ('RegistryKey', 'int'),
 ('RegistryValueName', 'int'),
 ('RegistryValueData', 'int'),
 ('ApplicationId', 'int'),
 ('ApplicationName', 'int'),
 ('OAuthApplicationId', 'int'),
 ('ThreatFamily', 'string'),
 ('FileName', 'int'),
 ('FolderPath', 'int'),
 ('ResourceIdName', 'int'),
 ('ResourceType', 'string'),
 ('Roles', 'string'),
 ('OSFamily', 'int'),
 ('OSVersion', '


 'Category', 'MitreTechniques', 'IncidentGrade', 'ActionGrouped',
 'ActionGranular', 'EntityType', 'EvidenceRole', 'ThreatFamily',
 'ResourceType', 'Roles', 'AntispamDirection', 'SuspicionLevel',
 'LastVerdict'

Now while these are the strings it seems like a lot of these even nominally int values are actually categoricals. Need to look into this.

In [ ]:
for f in df.columns:
  value = df.select(f).distinct().count()
  print(f, ": ", str(value), " unique values")

Id :  730778  unique values
OrgId :  5769  unique values
IncidentId :  466151  unique values
AlertId :  1265644  unique values
Timestamp :  760944  unique values
DetectorId :  8428  unique values
AlertTitle :  86149  unique values
Category :  20  unique values
MitreTechniques :  1194  unique values
IncidentGrade :  4  unique values
ActionGrouped :  4  unique values
ActionGranular :  17  unique values
EntityType :  33  unique values
EvidenceRole :  2  unique values
DeviceId :  75826  unique values
Sha256 :  106416  unique values
IpAddress :  285957  unique values
Url :  123252  unique values
AccountSid :  358401  unique values
AccountUpn :  530183  unique values
AccountObjectId :  343516  unique values
AccountName :  368250  unique values
DeviceName :  114541  unique values
NetworkMessageId :  375196  unique values
EmailClusterId :  26475  unique values
RegistryKey :  1341  unique values
RegistryValueName :  525  unique values
RegistryValueData :  699  unique values
ApplicationId :  172

## Test Set Inspection
Looking at basic features of test set to ensure no issues and verify any need for further data cleaning.


In [ ]:
df_test = spark.read.csv('/content/gdrive/MyDrive/GUIDE_Test.csv', header=True, inferSchema = True)
df_test.show(5)

+-------------+-----+----------+-------+-------------------+----------+----------+-----------------+--------------------+--------------+-------------+--------------+-----------------+------------+--------+------+---------+------+----------+----------+---------------+-----------+----------+----------------+--------------+-----------+-----------------+-----------------+-------------+---------------+------------------+------------+--------+----------+--------------+------------+----------+--------+---------+-----------------+--------------+-----------+-----------+-----+-----+-------+
|           Id|OrgId|IncidentId|AlertId|          Timestamp|DetectorId|AlertTitle|         Category|     MitreTechniques| IncidentGrade|ActionGrouped|ActionGranular|       EntityType|EvidenceRole|DeviceId|Sha256|IpAddress|   Url|AccountSid|AccountUpn|AccountObjectId|AccountName|DeviceName|NetworkMessageId|EmailClusterId|RegistryKey|RegistryValueName|RegistryValueData|ApplicationId|ApplicationName|OAuthApplica

In [ ]:
print('Data Shape: ', (df_test.count(), len(df_test.columns)))

Data Shape:  (4147992, 46)


In [ ]:
4147992/(9516837+4147992)

0.30355242645187874

The split of the data is roughly 70:30 train:test as mentioned by the authors of the dataset.

In [ ]:
df_test.printSchema()

root
 |-- Id: long (nullable = true)
 |-- OrgId: integer (nullable = true)
 |-- IncidentId: integer (nullable = true)
 |-- AlertId: integer (nullable = true)
 |-- Timestamp: timestamp (nullable = true)
 |-- DetectorId: integer (nullable = true)
 |-- AlertTitle: integer (nullable = true)
 |-- Category: string (nullable = true)
 |-- MitreTechniques: string (nullable = true)
 |-- IncidentGrade: string (nullable = true)
 |-- ActionGrouped: string (nullable = true)
 |-- ActionGranular: string (nullable = true)
 |-- EntityType: string (nullable = true)
 |-- EvidenceRole: string (nullable = true)
 |-- DeviceId: integer (nullable = true)
 |-- Sha256: integer (nullable = true)
 |-- IpAddress: integer (nullable = true)
 |-- Url: integer (nullable = true)
 |-- AccountSid: integer (nullable = true)
 |-- AccountUpn: integer (nullable = true)
 |-- AccountObjectId: integer (nullable = true)
 |-- AccountName: integer (nullable = true)
 |-- DeviceName: integer (nullable = true)
 |-- NetworkMessageId: i

The Usage column is unique to the test set and not relevant to our problem. I will simply be dropping this column.

In [ ]:
from pyspark.sql.functions import col, sum

null_counts = df_test.select([sum(col(c).isNull().cast("int")).alias(c) for c in df_test.columns])
null_counts.show()

+---+-----+----------+-------+---------+----------+----------+--------+---------------+-------------+-------------+--------------+----------+------------+--------+------+---------+---+----------+----------+---------------+-----------+----------+----------------+--------------+-----------+-----------------+-----------------+-------------+---------------+------------------+------------+--------+----------+--------------+------------+-------+--------+---------+-----------------+--------------+-----------+-----------+-----+----+-----+
| Id|OrgId|IncidentId|AlertId|Timestamp|DetectorId|AlertTitle|Category|MitreTechniques|IncidentGrade|ActionGrouped|ActionGranular|EntityType|EvidenceRole|DeviceId|Sha256|IpAddress|Url|AccountSid|AccountUpn|AccountObjectId|AccountName|DeviceName|NetworkMessageId|EmailClusterId|RegistryKey|RegistryValueName|RegistryValueData|ApplicationId|ApplicationName|OAuthApplicationId|ThreatFamily|FileName|FolderPath|ResourceIdName|ResourceType|  Roles|OSFamily|OSVersion|A

MitreTechniques, ActionGrouped, ActionGranular, EmailClusterId, ThreatFamily, ResourceType, Roles, AntispamDirection, SuspicionLevel, and LastVerdict all have missing values. These are, very convieniently, the same columns as the train set so I will be dropping them.

## Univariate Analyses

### Categoricals

- Category
- IncidentGrade
- ActionGrouped
- ActionGranular
- EvidenceRole
- ResourceType
- Roles
- OSFamily
- AntispamDirection
- SuspicionLevel
- LastVerdict
- EntityType
- OSVersion

In [ ]:
df.groupBy('Category').count().orderBy('count', ascending=False).show()

+-------------------+-------+
|           Category|  count|
+-------------------+-------+
|      InitialAccess|4293041|
|       Exfiltration|1577965|
| SuspiciousActivity|1003933|
|  CommandAndControl| 826691|
|             Impact| 750885|
|   CredentialAccess| 300518|
|          Execution| 267594|
|            Malware| 144081|
|          Discovery| 129342|
|        Persistence|  72682|
|     DefenseEvasion|  46662|
|    LateralMovement|  41754|
|         Ransomware|  18974|
|   UnwantedSoftware|  18211|
|         Collection|  14753|
|PrivilegeEscalation|   4671|
|            Exploit|   4648|
| CredentialStealing|    388|
|         WebExploit|     38|
|      Weaponization|      6|
+-------------------+-------+



In [ ]:
df.groupBy('IncidentGrade').count().orderBy('count', ascending=False).show()

+--------------+-------+
| IncidentGrade|  count|
+--------------+-------+
|BenignPositive|4110817|
|  TruePositive|3322713|
| FalsePositive|2031967|
|          NULL|  51340|
+--------------+-------+



In [ ]:
df.groupBy('ActionGrouped').count().orderBy('count', ascending=False).show(truncate=False)

+---------------------+-------+
|ActionGrouped        |count  |
+---------------------+-------+
|NULL                 |9460773|
|ContainAccount       |53760  |
|IsolateDevice        |2237   |
|Stop Virtual Machines|67     |
+---------------------+-------+



In [ ]:
df.groupBy('ActionGranular').count().orderBy('count', ascending=False).show(truncate=False)

+------------------------------------------+-------+
|ActionGranular                            |count  |
+------------------------------------------+-------+
|NULL                                      |9460773|
|update stsrefreshtokenvalidfrom timestamp.|21393  |
|account password changed                  |14059  |
|change user password.                     |13623  |
|isolateresponse                           |2043   |
|account disabled                          |1991   |
|disable account.                          |1143   |
|reset user password.                      |886    |
|forcepasswordresetremediation             |234    |
|quarantinefile                            |194    |
|msecidentitiesconfirmusercompromised      |146    |
|disableuser                               |107    |
|msecidentitiessuspenduser                 |81     |
|delete virtualmachines                    |67     |
|account deleted                           |55     |
|delete user.                              |35

In [ ]:
df.groupBy('EvidenceRole').count().orderBy('count', ascending=False).show()

+------------+-------+
|EvidenceRole|  count|
+------------+-------+
|     Related|5208644|
|    Impacted|4308193|
+------------+-------+



In [ ]:
df.groupBy('ResourceType').count().orderBy('count', ascending=False).show(26, truncate=False)

+------------------------------------+-------+
|ResourceType                        |count  |
+------------------------------------+-------+
|NULL                                |9509762|
|Virtual Machine                     |4146   |
|Key Vault                           |480    |
|App Service                         |404    |
|Azure Arc machine                   |373    |
|Storage Account                     |366    |
|Subscription                        |271    |
|SQL Database                        |226    |
|Azure Database for MySQL Server     |182    |
|Azure Resource                      |157    |
|Virtual Machine Scale Set           |112    |
|API Management Service              |94     |
|Key Vault Secret                    |77     |
|Azure Database for PostgreSQL Server|45     |
|SQL Server                          |41     |
|Azure Cosmos DB Account             |30     |
|Networking                          |14     |
|Synapse SQL Pool                    |11     |
|GCP Compute 

In [ ]:
df.groupBy('Roles').count().orderBy('count', ascending=False).show()

+--------------+-------+
|         Roles|  count|
+--------------+-------+
|          NULL|9298686|
|    Contextual| 130528|
|   Destination|  34324|
|    Suspicious|  28952|
|      Attacker|  13096|
|        Source|   7171|
|   Compromised|   3219|
|PolicyViolator|    666|
|      Attacked|    188|
|        Edited|      6|
|         Added|      1|
+--------------+-------+



In [ ]:
df.groupBy('OSFamily').count().orderBy('count', ascending=False).show()

+--------+-------+
|OSFamily|  count|
+--------+-------+
|       5|9322565|
|       0| 190036|
|       1|   2732|
|       2|   1496|
|       3|      7|
|       4|      1|
+--------+-------+



5 is overwhelming represented in this dataset. Is this sampling bias or reflective of targeting?

In [ ]:
df.groupBy('AntispamDirection').count().orderBy('count', ascending=False).show(truncate=False)

+--------------------------------------------------+-------+
|AntispamDirection                                 |count  |
+--------------------------------------------------+-------+
|NULL                                              |9339535|
|Inbound                                           |161111 |
|Intraorg                                          |15308  |
|Outbound                                          |868    |
|DomainPII_50d8b4a941c26b89482c94ab324b5a274f9ced66|10     |
|DomainPII_df80ab894e01e375bf55d12ba315c04029d3e32d|5      |
+--------------------------------------------------+-------+



In [ ]:
df.groupBy('SuspicionLevel').count().orderBy('count', ascending=False).show()

+--------------+-------+
|SuspicionLevel|  count|
+--------------+-------+
|          NULL|8072708|
|    Suspicious|1442614|
|  Incriminated|   1515|
+--------------+-------+



In [ ]:
df.groupBy('LastVerdict').count().orderBy('count', ascending=False).show(truncate=False)

+--------------------------------------------------+-------+
|LastVerdict                                       |count  |
+--------------------------------------------------+-------+
|NULL                                              |7282572|
|Suspicious                                        |1402997|
|Malicious                                         |433359 |
|NoThreatsFound                                    |397718 |
|DomainPII_50d8b4a941c26b89482c94ab324b5a274f9ced66|128    |
|DomainPII_9207384283ce115db5a590dd9ca5de21e5e99df2|63     |
+--------------------------------------------------+-------+



In [ ]:
df.groupBy('EntityType').count().orderBy('count', ascending=False).show(35,truncate=False)

+---------------------+-------+
|EntityType           |count  |
+---------------------+-------+
|Ip                   |2181194|
|User                 |1932416|
|MailMessage          |1173154|
|Machine              |699208 |
|File                 |688402 |
|Url                  |682578 |
|CloudLogonRequest    |638565 |
|Mailbox              |483158 |
|Process              |345732 |
|MailCluster          |224684 |
|CloudApplication     |216811 |
|CloudLogonSession    |212382 |
|RegistryValue        |11209  |
|AzureResource        |8166   |
|RegistryKey          |7277   |
|GenericEntity        |4294   |
|OAuthApplication     |2595   |
|Malware              |2580   |
|SecurityGroup        |1518   |
|BlobContainer        |306    |
|Blob                 |263    |
|MailboxConfiguration |226    |
|Nic                  |49     |
|IoTDevice            |31     |
|ActiveDirectoryDomain|11     |
|GoogleCloudResource  |10     |
|AmazonResource       |6      |
|Container            |2      |
|Kuberne

In [ ]:
df.groupBy('OSVersion').count().orderBy('count', ascending=False).show(70)

+---------+-------+
|OSVersion|  count|
+---------+-------+
|       66|9322572|
|        0| 187405|
|        2|   1892|
|        1|   1652|
|        3|   1125|
|        4|    732|
|        6|    362|
|        5|    266|
|        8|    132|
|        9|    109|
|       10|     89|
|        7|     81|
|       11|     66|
|       12|     54|
|       16|     38|
|       13|     32|
|       14|     31|
|       15|     22|
|       19|     18|
|       17|     17|
|       20|     16|
|       22|     15|
|       21|     14|
|       24|     13|
|       25|      7|
|       26|      7|
|       27|      6|
|       31|      5|
|       28|      5|
|       33|      4|
|       35|      4|
|       38|      3|
|       41|      3|
|       29|      3|
|       42|      3|
|       37|      3|
|       40|      3|
|       46|      2|
|       34|      2|
|       45|      2|
|       30|      2|
|       43|      2|
|       23|      2|
|       44|      2|
|       47|      1|
|       52|      1|
|       62|      1|


OSVersion 66 is overwhelmingly represented here. As with OS could refelct sampling bias or actual target selection.

In [ ]:
df.groupBy('ThreatFamily').count().orderBy('count', ascending=False).show(10, truncate=False)

+-------------------------+-------+
|ThreatFamily             |count  |
+-------------------------+-------+
|NULL                     |9441956|
|Phish                    |3526   |
|Malgent                  |2636   |
|Donoff                   |2519   |
|CustomEnterpriseBlock    |2145   |
|CustomEnterpriseBlockOnly|2139   |
|CymRan                   |1648   |
|CustomCertEnterpriseBlock|1595   |
|Phonzy                   |1493   |
|Wacatac                  |1185   |
+-------------------------+-------+
only showing top 10 rows



In [ ]:
df.drop('MitreTechniques', 'ActionGrouped', 'ActionGranular', 'EmailClusterId', 'ThreatFamily', 'ResourceType', 'Roles', 'AntispamDirection', 'SuspicionLevel', 'LastVerdict')

+-------------+-----+----------+-------+-------------------+----------+----------+------------------+--------------+----------------+------------+--------+------+---------+------+----------+----------+---------------+-----------+----------+----------------+-----------+-----------------+-----------------+-------------+---------------+------------------+--------+----------+--------------+--------+---------+-----------+-----+-----+
|           Id|OrgId|IncidentId|AlertId|          Timestamp|DetectorId|AlertTitle|          Category| IncidentGrade|      EntityType|EvidenceRole|DeviceId|Sha256|IpAddress|   Url|AccountSid|AccountUpn|AccountObjectId|AccountName|DeviceName|NetworkMessageId|RegistryKey|RegistryValueName|RegistryValueData|ApplicationId|ApplicationName|OAuthApplicationId|FileName|FolderPath|ResourceIdName|OSFamily|OSVersion|CountryCode|State| City|
+-------------+-----+----------+-------+-------------------+----------+----------+------------------+--------------+----------------+-

In [ ]:
df.groupBy('RegistryKey').count().orderBy('count', ascending=False).show()

+-----------+-------+
|RegistryKey|  count|
+-----------+-------+
|       1631|9499313|
|          0|  11076|
|          1|    245|
|          3|    235|
|          6|    141|
|          4|    136|
|          5|    131|
|          8|    118|
|          7|     98|
|         12|     75|
|          9|     70|
|         11|     66|
|         10|     66|
|         15|     58|
|          2|     53|
|         13|     44|
|         16|     40|
|         17|     36|
|         24|     36|
|         25|     34|
+-----------+-------+
only showing top 20 rows



### Numericals

In [ ]:
df.columns

['Id',
 'OrgId',
 'IncidentId',
 'AlertId',
 'Timestamp',
 'DetectorId',
 'AlertTitle',
 'Category',
 'MitreTechniques',
 'IncidentGrade',
 'ActionGrouped',
 'ActionGranular',
 'EntityType',
 'EvidenceRole',
 'DeviceId',
 'Sha256',
 'IpAddress',
 'Url',
 'AccountSid',
 'AccountUpn',
 'AccountObjectId',
 'AccountName',
 'DeviceName',
 'NetworkMessageId',
 'EmailClusterId',
 'RegistryKey',
 'RegistryValueName',
 'RegistryValueData',
 'ApplicationId',
 'ApplicationName',
 'OAuthApplicationId',
 'ThreatFamily',
 'FileName',
 'FolderPath',
 'ResourceIdName',
 'ResourceType',
 'Roles',
 'OSFamily',
 'OSVersion',
 'AntispamDirection',
 'SuspicionLevel',
 'LastVerdict',
 'CountryCode',
 'State',
 'City']

In [ ]:
num_col = ['Id', 'OrgId', 'IncidentId', 'AlertId', 'DetectorId', 'AlertTitle',
            'DeviceId', 'Sha256', 'IpAddress', 'Url', 'AccountSid',
            'AccountUpn', 'AccountObjectId', 'AccountName', 'DeviceName',
            'NetworkMessageId', 'RegistryKey', 'RegistryValueName',
            'RegistryValueData', 'ApplicationId', 'EmailClusterId',
            'ApplicationName', 'OAuthApplicationId', 'ThreatFamily', 'FileName',
            'FolderPath', 'ResourceIdName', 'CountryCode', 'State', 'City']

In [ ]:
len(num_col)

30

In [ ]:
df.select("OrgId").summary().show(truncate=False)

+-------+------------------+
|summary|OrgId             |
+-------+------------------+
|count  |9516837           |
|mean   |181.57995014520054|
|stddev |386.77837703936484|
|min    |0                 |
|25%    |10                |
|50%    |45                |
|75%    |171               |
|max    |6147              |
+-------+------------------+



In [ ]:
for c in num_col:
  df.select(c).summary().show(truncate=False)

+-------+---------------------+
|summary|Id                   |
+-------+---------------------+
|count  |9516837              |
|mean   |8.425494025715468E11 |
|stddev |4.9624987503972345E11|
|min    |0                    |
|25%    |412316863917         |
|50%    |841813590289         |
|75%    |1271310322153        |
|max    |1709396988938        |
+-------+---------------------+

+-------+------------------+
|summary|OrgId             |
+-------+------------------+
|count  |9516837           |
|mean   |181.57995014520054|
|stddev |386.77837703936484|
|min    |0                 |
|25%    |10                |
|50%    |45                |
|75%    |171               |
|max    |6147              |
+-------+------------------+

+-------+------------------+
|summary|IncidentId        |
+-------+------------------+
|count  |9516837           |
|mean   |70663.48631630446 |
|stddev |120836.85320205832|
|min    |0                 |
|25%    |504               |
|50%    |10335             |
|75% 

Inspecting the output we see most features are HEAVILY skewed.

## Time Feature and Transformation
 Should probably move this so time is part of the transformation pipeline. For now though looking at this to get an understanding of time.



In [ ]:
from pyspark.sql.functions import year, month, dayofmonth, hour, minute, second, dayofweek, weekofyear

df_time = df.withColumn("year", year(df["Timestamp"])) \
       .withColumn("month", month(df["Timestamp"])) \
       .withColumn("day", dayofmonth(df["Timestamp"])) \
       .withColumn("hour", hour(df["Timestamp"])) \
       .withColumn("minute", minute(df["Timestamp"])) \
       .withColumn("second", second(df["Timestamp"])) \
       .withColumn("day_of_week", dayofweek(df["Timestamp"])) \
       .withColumn("week_of_year", weekofyear(df["Timestamp"]))
df_time.show(5)

+-------------+-----+----------+-------+-------------------+----------+----------+-----------------+---------------+--------------+-------------+--------------+----------+------------+--------+------+---------+------+----------+----------+---------------+-----------+----------+----------------+--------------+-----------+-----------------+-----------------+-------------+---------------+------------------+------------+--------+----------+--------------+------------+-----+--------+---------+-----------------+--------------+-----------+-----------+-----+-----+----+-----+---+----+------+------+-----------+------------+
|           Id|OrgId|IncidentId|AlertId|          Timestamp|DetectorId|AlertTitle|         Category|MitreTechniques| IncidentGrade|ActionGrouped|ActionGranular|EntityType|EvidenceRole|DeviceId|Sha256|IpAddress|   Url|AccountSid|AccountUpn|AccountObjectId|AccountName|DeviceName|NetworkMessageId|EmailClusterId|RegistryKey|RegistryValueName|RegistryValueData|ApplicationId|Applic

In [ ]:
df_time.groupBy('year').count().orderBy('count', ascending=False).show()

+----+-------+
|year|  count|
+----+-------+
|2024|9516658|
|2023|    179|
+----+-------+



In [ ]:
df_time.groupBy('month').count().orderBy('count', ascending=False).show()

+-----+-------+
|month|  count|
+-----+-------+
|    6|8662447|
|    5| 853082|
|    1|    310|
|    2|    297|
|    4|    287|
|    3|    235|
|   12|    168|
|   11|     11|
+-----+-------+



In [ ]:
df_time.groupBy('day').count().orderBy('count', ascending=False).show()

+---+------+
|day| count|
+---+------+
|  4|910667|
|  5|899745|
|  3|784976|
| 11|742607|
|  6|736842|
| 10|698534|
| 12|683131|
| 13|660391|
|  7|630698|
|  9|475598|
| 14|469160|
|  8|452636|
| 15|176232|
| 16|145826|
| 26|113551|
|  2| 99237|
| 24| 98830|
| 25| 98512|
| 23| 84361|
| 31| 81432|
+---+------+
only showing top 20 rows



In [ ]:
df_time.groupBy('week_of_year').count().orderBy('count', ascending=False).show()

+------------+-------+
|week_of_year|  count|
+------------+-------+
|          23|4890537|
|          24|3575067|
|          21| 579401|
|          22| 427695|
|          25|  41631|
|          20|    815|
|          19|    312|
|           6|    153|
|          18|     94|
|           5|     87|
|          14|     85|
|          16|     79|
|          52|     71|
|          13|     65|
|           1|     64|
|           3|     64|
|          10|     63|
|           2|     63|
|           9|     60|
|           4|     59|
+------------+-------+
only showing top 20 rows



In [ ]:
(4890537 + 3575067 + 579401 + 427695 + 41631)/df_time.count()

0.9997366772174411

An overwhelming majority of the data occurs between week 21- 25.  This may be indicitive of either extreme sampling bias or a significant cyber attack campaign in this time period. For the final report, this necessitates further study.

In [ ]:
df_time.groupBy('hour').count().orderBy('count', ascending=False).show(25)

+----+------+
|hour| count|
+----+------+
|  15|482147|
|  17|479501|
|  16|478842|
|  18|470807|
|  13|451389|
|  14|450973|
|  19|448513|
|  20|434794|
|  12|421560|
|  21|417091|
|  11|397688|
|  22|396944|
|  10|378357|
|  23|377580|
|   1|377563|
|   9|372211|
|   0|354859|
|   8|347197|
|   2|344377|
|   3|335596|
|   5|326636|
|   7|325715|
|   4|324403|
|   6|322094|
+----+------+



We also see a very heavy bias towards 2024 and June. I'll still be using these time features as having them tends to be better for data but still its something to be aware of how biased it appears in sampling.

## Mitre Techniques Analysis

While we will be removing this column due to the volume of missing values, inspecting it is likely to pay dividends in understanding the nature of the most common attacks in the dataset.

In [ ]:
mitre_count = df.groupBy('MitreTechniques').count().orderBy('count', ascending=False).show(11, truncate=False)

+-------------------------+-------+
|MitreTechniques          |count  |
+-------------------------+-------+
|NULL                     |5468386|
|T1078;T1078.004          |1354904|
|T1566.002                |814308 |
|T1566                    |659591 |
|T1133                    |145579 |
|T1566.001                |136892 |
|T1110;T1110.003;T1110.001|88661  |
|T1087;T1087.002          |54564  |
|T1110                    |41768  |
|T1078;T1098              |40342  |
|T1559;T1106;T1059.005    |37869  |
+-------------------------+-------+
only showing top 11 rows



In [ ]:
mitre_count_nonull = df.filter(df.MitreTechniques.isNotNull()).groupBy('MitreTechniques').count().orderBy('count', ascending=False).show(10, truncate=False)

+-------------------------+-------+
|MitreTechniques          |count  |
+-------------------------+-------+
|T1078;T1078.004          |1354904|
|T1566.002                |814308 |
|T1566                    |659591 |
|T1133                    |145579 |
|T1566.001                |136892 |
|T1110;T1110.003;T1110.001|88661  |
|T1087;T1087.002          |54564  |
|T1110                    |41768  |
|T1078;T1098              |40342  |
|T1559;T1106;T1059.005    |37869  |
+-------------------------+-------+
only showing top 10 rows



In [ ]:
from pyspark.sql.functions import col, count

top_attacks = (
    df.filter(df.MitreTechniques.isNotNull())
    .groupBy("MitreTechniques")
    .agg(count("*").alias("count"))
    .orderBy(col("count").desc())
    .limit(10)
)

df_filtered = df.join(top_attacks, on="MitreTechniques", how="inner")
df_filtered.stat.crosstab("MitreTechniques", "IncidentGrade").show(truncate=False)

+-----------------------------+--------------+-------------+------------+----+
|MitreTechniques_IncidentGrade|BenignPositive|FalsePositive|TruePositive|null|
+-----------------------------+--------------+-------------+------------+----+
|T1078;T1078.004              |35897         |308347       |1009935     |725 |
|T1087;T1087.002              |23935         |21831        |8782        |16  |
|T1566                        |346566        |167763       |141964      |3298|
|T1110;T1110.003;T1110.001    |1549          |3102         |84002       |8   |
|T1133                        |19124         |126455       |0           |0   |
|T1559;T1106;T1059.005        |37869         |0            |0           |0   |
|T1566.001                    |90698         |7398         |38792       |4   |
|T1566.002                    |391993        |128955       |292731      |629 |
|T1078;T1098                  |35268         |3721         |1341        |12  |
|T1110                        |24047         |7081  

I will be looking into these further for the final report.